In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Primer ejemplo") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [6]:
flightData2015 = spark\
.read\
.option("inferSchema", "true")\
.option("header", "true")\
.csv("../../data/flight-data/csv/2015-summary.csv")

In [7]:
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [8]:
flightData2015.sort("count").explain()

== Physical Plan ==
*(1) Sort [count#40 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#40 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#51]
   +- FileScan csv [DEST_COUNTRY_NAME#38,ORIGIN_COUNTRY_NAME#39,count#40] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/media/jose/Repositorio/Git_Hub/Big-Data-Procesing/Spark-The-Definitive-Gu..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [9]:
spark.conf.set("spark.sql.shuffle.partitions", "5")
flightData2015.sort("count").take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [10]:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [12]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

dataFrameWay = flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.count()

sqlWay.explain()
dataFrameWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#38], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#38, 5), ENSURE_REQUIREMENTS, [id=#118]
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#38], functions=[partial_count(1)])
      +- FileScan csv [DEST_COUNTRY_NAME#38] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/media/jose/Repositorio/Git_Hub/Big-Data-Procesing/Spark-The-Definitive-Gu..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#38], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#38, 5), ENSURE_REQUIREMENTS, [id=#137]
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#38], functions=[partial_count(1)])
      +- FileScan csv [DEST_COUNTRY_NAME#38] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/media/jose/Repositorio/Git_Hub/Big-Data-Procesing/Spark-The-Defi

In [13]:
spark.sql("SELECT max(count) from flight_data_2015").take(1)

[Row(max(count)=370002)]

In [14]:
from pyspark.sql.functions import max
flightData2015.select(max("count")).take(1)

[Row(max(count)=370002)]

In [15]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")
maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [16]:
from pyspark.sql.functions import desc
flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)\
.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [17]:
flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)\
.explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#141L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#38,destination_total#141L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#38], functions=[sum(cast(count#40 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#38, 5), ENSURE_REQUIREMENTS, [id=#264]
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#38], functions=[partial_sum(cast(count#40 as bigint))])
         +- FileScan csv [DEST_COUNTRY_NAME#38,count#40] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/media/jose/Repositorio/Git_Hub/Big-Data-Procesing/Spark-The-Definitive-Gu..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


